In [43]:
import pandas as pd
import numpy as np
import json
import logging

In [77]:
primary_domain_dict = {
    "metric":[
        {"metric1":{
            "d_avg": 0,
            "d_count": 0,
            "h_avg": 0,
            "h_count": 0,
            "last_h_sending_timestamp": 0,
            "last_d_sending_timestamp": 0,
         }
        },
        {"metric2":{
             "d_avg": 0,
             "d_count": 0,
             "h_avg": 0,
             "h_count": 0,
             "last_h_sending_timestamp": 0,
             "last_d_sending_timestamp": 0,
             }
        },
        {"metric3":{
             "d_avg": 0,
             "d_count": 0,
             "h_avg": 0,
             "h_count": 0,
             "last_h_sending_timestamp": 0,
             "last_d_sending_timestamp": 0,
         }
        },
    ],

}

数据每min计算一次，并更新primary_domain_dict，每小时和每天发送至Cassandra数据库。

In [78]:
from digital_machine.runtime.default.timestamps import _align_timestamp

# 每30min计算一次
START = 1577808000
TIMEZONE = "Asia/Shanghai"

H_count = []
H_sum = []
D_count = []
D_sum = []

for i in range(55):
    timestamp = START + 30 * 60 * i
    timestamp_1h = _align_timestamp(timestamp, TIMEZONE, resolution = "1h")
    timestamp_1d = _align_timestamp(timestamp, TIMEZONE, resolution = "1d")
    
    for i, d in enumerate(primary_domain_dict["metric"]):
        
        exec(f"{list(d.keys())[0]} = np.random.randint(100)")
        
        metric_name = list(d.keys())[0]
        
        # 更新h_count，h_avg，d_count，d_avg
        h_count = primary_domain_dict["metric"][i][metric_name]["h_count"]
        h_avg = primary_domain_dict["metric"][i][metric_name]["h_avg"]
        d_count = primary_domain_dict["metric"][i][metric_name]["d_count"]
        d_avg = primary_domain_dict["metric"][i][metric_name]["d_avg"]
        
        last_d_sending_timestamp = primary_domain_dict["metric"][i][metric_name]["last_d_sending_timestamp"]
        last_h_sending_timestamp = primary_domain_dict["metric"][i][metric_name]["last_h_sending_timestamp"]
        
        # if每日0点，先发送数据，再更新数据
        # TODO 小时、天的数据write_timeseries到数据库，并清空 h_count，h_avg，d_count，d_avg
        if last_d_sending_timestamp < timestamp_1d :
#             logging.error("last_d_sending_timestamp=%s, timestamp_1d=%s",last_d_sending_timestamp, timestamp_1d)
            D_count.append(d_count)
            D_sum.append(d_avg * d_count)
            H_count.append(h_count)
            H_sum.append(h_avg * h_count)

            h_count, h_avg, d_count, d_avg = 0, 0, 0, 0

            primary_domain_dict["metric"][i][metric_name]["last_d_sending_timestamp"] = timestamp
            logging.error("metric_name:: %s, sending d data in timestamp_1d=%s", metric_name, timestamp_1d)
            
            primary_domain_dict["metric"][i][metric_name]["last_h_sending_timestamp"] = timestamp
            logging.error("metric_name:: %s, sending h data in timestamp_1h=%s", metric_name, timestamp_1h)
            
        # elif 不是每日0点，但是是整小时点
        # TODO 小时的数据write_timeseries到数据库，并清空 h_count，h_avg
        elif last_h_sending_timestamp < timestamp_1h:
#             logging.error("last_h_sending_timestamp=%s, timestamp_1h=%s",last_h_sending_timestamp, timestamp_1h)
            H_count.append(h_count)
            H_sum.append(h_avg * h_count)

            h_count, h_avg = 0, 0

            primary_domain_dict["metric"][i][metric_name]["last_h_sending_timestamp"] = timestamp
            logging.error("metric_name:: %s, sending h data in timestamp_1h=%s", metric_name, timestamp_1h)
        
        # 两种情况都不是，则更新数据
        logging.error("metric_name::%s update h_avg, h_count", metric_name)
        h_avg = (h_avg * h_count + eval(f"{metric_name}")) / (h_count + 1)
        h_count += 1
        
        logging.error("metric_name::%s update d_avg, d_count", metric_name)
        d_avg = (d_avg * d_count + eval(f"{metric_name}")) / (d_count + 1)
        d_count += 1

        # 回写mango
        primary_domain_dict["metric"][i][metric_name]["h_count"] = h_count
        primary_domain_dict["metric"][i][metric_name]["h_avg"] = h_avg
        primary_domain_dict["metric"][i][metric_name]["d_count"] = d_count
        primary_domain_dict["metric"][i][metric_name]["d_avg"] = d_avg

ERROR:root:metric_name:: metric1, sending d data in timestamp_1d=1577808000
ERROR:root:metric_name:: metric1, sending h data in timestamp_1h=1577808000
ERROR:root:metric_name::metric1 update h_avg, h_count
ERROR:root:metric_name::metric1 update d_avg, d_count
ERROR:root:metric_name:: metric2, sending d data in timestamp_1d=1577808000
ERROR:root:metric_name:: metric2, sending h data in timestamp_1h=1577808000
ERROR:root:metric_name::metric2 update h_avg, h_count
ERROR:root:metric_name::metric2 update d_avg, d_count
ERROR:root:metric_name:: metric3, sending d data in timestamp_1d=1577808000
ERROR:root:metric_name:: metric3, sending h data in timestamp_1h=1577808000
ERROR:root:metric_name::metric3 update h_avg, h_count
ERROR:root:metric_name::metric3 update d_avg, d_count
ERROR:root:metric_name::metric1 update h_avg, h_count
ERROR:root:metric_name::metric1 update d_avg, d_count
ERROR:root:metric_name::metric2 update h_avg, h_count
ERROR:root:metric_name::metric2 update d_avg, d_count
ERRO

In [79]:
primary_domain_dict

{'metric': [{'metric1': {'d_avg': 60.57142857142857,
    'd_count': 7,
    'h_avg': 91.0,
    'h_count': 1,
    'last_h_sending_timestamp': 1577905200,
    'last_d_sending_timestamp': 1577894400}},
  {'metric2': {'d_avg': 72.14285714285714,
    'd_count': 7,
    'h_avg': 78.0,
    'h_count': 1,
    'last_h_sending_timestamp': 1577905200,
    'last_d_sending_timestamp': 1577894400}},
  {'metric3': {'d_avg': 59.42857142857143,
    'd_count': 7,
    'h_avg': 19.0,
    'h_count': 1,
    'last_h_sending_timestamp': 1577905200,
    'last_d_sending_timestamp': 1577894400}}]}